In [1]:
import os

In [6]:
from instagram.client import InstagramAPI

INSTAGRAM_CLIENT_ID = os.environ['INSTAGRAM_DD_CLIENT_ID']
INSTAGRAM_CLIENT_SECRET = os.environ['INSTAGRAM_DD_CLIENT_SECRET']
INSTAGRAM_ACCESS_TOKEN = os.environ['INSTAGRAM_DD_ACCESS_TOKEN']
client_ip = '127.0.0.1'
api = InstagramAPI(client_id=INSTAGRAM_CLIENT_ID, 
                   client_secret=INSTAGRAM_CLIENT_SECRET, 
                   client_ips=client_ip,
                   access_token=INSTAGRAM_ACCESS_TOKEN)

In [7]:
def pullMediaByUser(user_id):
    media, next_ = api.user_recent_media(user_id=user_id)
    while next_:
        more_media, next_ = api.user_recent_media(with_next_url=next_)
        media.extend(more_media)
        
    return media

In [8]:
def pullMediaByHashtag(tag_name, count=100):
    media, next_ = api.tag_recent_media(tag_name=tag_name)
    while next_:
        more_media, next_ = api.tag_recent_media(with_next_url=next_, tag_name=tag_name)
        media.extend(more_media)
        
        if len(media) >= count:
            break
            
        if len(media) % 1000 == 0:
            print (len(media))
        
    return media

In [9]:
hashtagToPull = 'wakenbake'

In [10]:
recent_media = pullMediaByHashtag(hashtagToPull, 50000)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000


In [11]:
len(recent_media)

50014

In [12]:
## see what's in the data
print (dir(recent_media[1]))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__unicode__', '__weakref__', 'caption', 'comment_count', 'comments', 'created_time', 'filter', 'get_low_resolution_url', 'get_standard_resolution_url', 'get_thumbnail_url', 'id', 'images', 'like_count', 'likes', 'link', 'object_from_dictionary', 'tags', 'type', 'user', 'user_has_liked', 'users_in_photo']


In [15]:
data = []
for media in recent_media:
    item = {}

    ## user details
    item['username'] = media.user.username
    item['user_id'] = media.user.id
    
    ## media details
    item['id'] = media.id
    item['type'] = media.type
    item['filter'] = media.filter
    item['created_time'] = media.created_time
    
    ## check if there is a caption
    item['caption'] = ''
    if media.caption != None:
        item['caption'] = media.caption.text
    
    ## parse tags
    item['tags'] = []
    for tag in media.tags:
        item['tags'].append(tag.name)
    
    ## parse comments
    item['comment_count'] = media.comment_count
    item['comments'] = []
    for comment in media.comments:
        entry = {}
        entry['username'] = comment.user.username
        entry['user_id'] = comment.user.id
        entry['created_time'] = comment.created_at
        entry['text'] = comment.text
        item['comments'].append(entry)
    
    ## parse like information
    item['like_count'] = media.like_count
    ## adjust for liking own picture
    if media.user_has_liked == True:
        item['like_count'] -= 1
    
    ## parse location if available
    item['location'] = {}
    if 'location' in dir(media):
        if 'latitude' in dir(media.location.point):
            item['location']['latitude'] = media.location.point.latitude
            item['location']['longitude'] = media.location.point.longitude
    
    data.append(item)

In [23]:
## see what's in there

print(data[0])
print ('')
print(data[1])
print ('')
print(data[2])

{'tags': ['highsociety', 'plant', 'stoner', 'stoners', 'gethigh', 'instahigh', 'marijuana', 'cannabisculture', 'staylifted', 'cannabiscommunity', 'stoned', 'weshouldsmoke', 'pothead', 'fueledbythc', 'weedstagram', 'potheadsociety', 'hightimes', 'getlifted', 'potheads', 'calistoner', 'instaweed', 'letssmoke', 'cannabis', 'stayhigh', 'dank', 'highaf', 'blowingclouds', 'wakenbake', 'highlife'], 'username': 'mystical_stoner', 'comment_count': 1, 'comments': [{'text': '#Pothead #Stoner #Stoners #highaf #staylifted #cannabis #gethigh #marijuana #Calistoner #stayhigh #Weshouldsmoke #instaweed #stoned #cannabiscommunity #cannabisculture #potheadsociety #fueledbythc #getlifted #potheads #letssmoke #highsociety #instahigh #potheads #plant #highlife #hightimes #wakenbake #dank #weedstagram #blowingclouds', 'user_id': '1621796574', 'username': 'mystical_stoner', 'created_time': datetime.datetime(2015, 8, 14, 18, 56, 24)}], 'location': {}, 'type': 'image', 'created_time': datetime.datetime(2015, 8,

In [28]:
for item in data[:10]:
    print(item['created_time'])

2015-08-14 18:56:14
2015-08-14 18:55:17
2015-08-14 18:54:44
2015-08-14 18:54:06
2015-08-14 18:53:20
2015-08-14 18:52:25
2015-08-14 18:50:53
2015-08-14 18:50:53
2015-08-14 18:50:29
2015-08-14 18:49:15


In [29]:
for item in data[-10:]:
    print(item['created_time'])

2015-06-18 10:56:24
2015-06-18 10:51:45
2015-06-18 10:51:44
2015-06-18 10:49:40
2015-06-18 10:48:50
2015-06-18 10:46:56
2015-06-18 10:45:17
2015-06-18 10:44:23
2015-06-18 10:40:37
2015-06-18 10:38:55


In [30]:
## throw into mongo
from pymongo import MongoClient

In [31]:
client = MongoClient('mongodb://localhost:27017/')
db = client['instagram']
collectionName = hashtagToPull + '-tags'
collection = db[collectionName]

In [32]:
## clear what was in there before
collection.drop()

In [33]:
result = collection.insert_many(data)
len(result.inserted_ids)

50014